In [1]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import Module
import numpy as np
from torch.nn._functions.quantize import quantize
from torch.nn import functional as F


import torch.utils.data
import torchvision
from torchvision import datasets, transforms
import torch.optim as optim

from torch.nn.init import kaiming_normal
from torch.nn.init import xavier_normal

In [2]:
def kaiming_normal_quantized(tensor, bits, a=0, mode="fan_in"):
    kaiming_normal(tensor, a, mode)
    tensor = quantize(tensor, bits)
def xavier_normal_quantized(tensor, bits):
    xavier_normal(tensor)
    tensor = quantize(tensor, bits)

In [3]:
class Net(Module):
    def __init__(self, forward_bits, backward_bits, mode="Stochastic"):
        super(Net, self).__init__()
        self.Q = nn.Quantize(forward_bits, backward_bits, mode)
        self.conv1 = nn.Conv2d(1, 8, kernel_size=5)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=5)
        self.fc1 = nn.Linear(4*4*16, 128)
        self.fc2 = nn.Linear(128, 10)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                xavier_normal_quantized(m.weight.data, bits)
    def forward(self, x):
        out = F.relu(F.max_pool2d(self.conv1(x), 2))
        out = self.Q(out)
        out = F.relu(F.max_pool2d(self.conv2(out), 2))
        out = self.Q(out)
        out = out.view(-1, 4*4*16)
        out = F.relu(self.fc1(out))
        out = self.Q(out)
        out = F.relu(self.fc2(out))
        out = self.Q(out)
        return F.log_softmax(out, dim=1)

In [4]:
bath_size = 100
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=bath_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=bath_size, shuffle=True)

In [5]:
bits = (16, 12)
model = Net((16, 10), (16, 14))

optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0005)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0,95, last_epoch=-1)

def train(epoch):
#     scheduler.step()
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
            

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 11):
    train(epoch)
    test()


/Users/tianyizhang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.281325
Train Epoch: 1 [10000/60000 (17%)]	Loss: 1.373806
Train Epoch: 1 [20000/60000 (33%)]	Loss: 1.558008
Train Epoch: 1 [30000/60000 (50%)]	Loss: 1.117014
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.932492
Train Epoch: 1 [50000/60000 (83%)]	Loss: 1.044525


/Users/tianyizhang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/Users/tianyizhang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 0.8119, Accuracy: 6774/10000 (67%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.839396
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.867052
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.474616


KeyboardInterrupt: 